# 机器翻译

将一段文本自动翻译到另一种语言。因为一段文本序列在不同语言的长度不一定相同，所以我们使用机器翻译为例介绍编码器-解码器和注意力机制的运用。

i am chinese --> 我 是 中 国 人 3个字符映射为5个字符

## Sequence to sequence模型
训练：
![](https://cdn.kesci.com/upload/image/q5jc7a53pt.png?imageView2/0/w/640/h/640)

encoder的输出的语义编码作为decoder隐藏状态$H_{-1}$。decoder就是一个生成式的语言模型。

测试：

![](https://cdn.kesci.com/upload/image/q5jcecxcba.png?imageView2/0/w/640/h/640)

具体结构：

![](https://cdn.kesci.com/upload/image/q5jccjhkii.png?imageView2/0/w/500/h/500)

encoder可以是n层的循环神经网络，source输入，需要转化为词向量。hello --> 词向量；word --> 词向量。

```python 
class Seq2SeqEncoder(d2l.Encoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        self.num_hiddens=num_hiddens
        self.num_layers=num_layers
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size,num_hiddens, num_layers, dropout=dropout)
   
    def begin_state(self, batch_size, device):
        return [torch.zeros(size=(self.num_layers, batch_size, self.num_hiddens),  device=device),
                torch.zeros(size=(self.num_layers, batch_size, self.num_hiddens),  device=device)]
    def forward(self, X, *args):
        X = self.embedding(X) # X shape: (batch_size, seq_len, embed_size)
        X = X.transpose(0, 1)  # RNN needs first axes to be time
        # state = self.begin_state(X.shape[1], device=X.device)
        out, state = self.rnn(X)
        # The shape of out is (seq_len, batch_size, num_hiddens).
        # state contains the hidden state and the memory cell
        # of the last time step, the shape is (num_layers, batch_size, num_hiddens)
        return out, state
```

decoder：

```python
class Seq2SeqDecoder(d2l.Decoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size,num_hiddens, num_layers, dropout=dropout)
        self.dense = nn.Linear(num_hiddens,vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        X = self.embedding(X).transpose(0, 1)
        out, state = self.rnn(X, state)
        # Make the batch to be the first dimension to simplify loss computation.
        out = self.dense(out).transpose(0, 1)
        return out, state
    
```

多了一个dense layer，这是为了产生输出，注意vocab_size是整个单词表的个数。

## Beam search

![](https://cdn.kesci.com/upload/image/q5jchqoppn.png?imageView2/0/w/440/h/440)

![](https://cdn.kesci.com/upload/image/q5jcia86z1.png?imageView2/0/w/640/h/640)

# 注意力机制

在Encoder-Decoder结构中，Encoder把所有的输入序列都编码成一个统一的语义特征c再解码，因此， c中必须包含原始序列中的所有信息，它的长度就成了限制模型性能的瓶颈。如机器翻译问题，当要翻译的句子较长时，一个c可能存不下那么多信息，就会造成翻译精度的下降。


Attention机制通过在每个时间输入不同的c来解决这个问题，下图是带有Attention机制的Decoder：

![](https://pic2.zhimg.com/80/v2-8da16d429d33b0f2705e47af98e66579_hd.jpg)

每一个c会自动去选取与当前所要输出的y最合适的上下文信息。具体来说，我们用 $a_{ij}$ 衡量Encoder中第j阶段的hj和解码时第i阶段的相关性，最终Decoder中第i阶段的输入的上下文信息$c_{i}$就来自于所有hj对$a_{ij}$的加权和。

以机器翻译为例：

![](https://pic1.zhimg.com/80/v2-d266bf48a1d77e7e4db607978574c9fc_hd.jpg)

输入的序列是“我爱中国”，因此，Encoder中的h1、h2、h3、h4就可以分别看做是“我”、“爱”、“中”、“国”所代表的信息。在翻译成英语时，第一个上下文c1应该和“我”这个字最相关，因此对应的a11就比较大，而相应的 a12 、 a13 、 a14 就比较小。c2应该和“爱”最相关，因此对应的 a22 就比较大。最后的c3和h3、h4最相关，因此 a33 、 a34 的值就比较大。

至此，关于Attention模型，我们就只剩最后一个问题了，那就是：这些权重aij是怎么来的？

事实上， aij同样是从模型中学出的，它实际和Decoder的第i-1阶段的隐状态、Encoder第j个阶段的隐状态有关。

同样还是拿上面的机器翻译举例， a1j 的计算（此时箭头就表示对h'和 hj 同时做变换）：


![](https://pic4.zhimg.com/80/v2-5561fa61321f31113043fb9711ee3263_hd.jpg)



a2j 的计算：

![](https://pic1.zhimg.com/80/v2-50473aa7b1c20d680abf8ca36d82c9e4_hd.jpg)


a3j 的计算：

![](https://pic4.zhimg.com/80/v2-07f7411c77901a7bd913e55884057a63_hd.jpg)

以上就是带有Attention的Encoder-Decoder模型计算的全过程。

### Attention机制的本质思想

我们可以这样来看待Attention机制（参考图9）：将Source中的构成元素想象成是由一系列的<Key,Value>数据对构成，此时给定Target中的某个元素Query，通过计算Query和各个Key的相似性或者相关性，得到每个Key对应Value的权重系数，然后对Value进行加权求和，即得到了最终的Attention数值。所以本质上Attention机制是对Source中元素的Value值进行加权求和，而Query和Key用来计算对应Value的权重系数。即可以将其本质思想改写为如下公式：

$Attention(Query, Source) = \sum_{i=1}^L f(Query, Key_i) *Value_i$

![](https://pic3.zhimg.com/80/v2-5adc62ac92667bac12287f349f1c8cc6_hd.jpg)

其中L表示Source的长度，这个和之前机器翻译时的公式是一样的，只是在机器翻译时，Key和Value是一样的，都是Source中每个时刻隐藏层状态。

#### Attention机制的具体计算过程



和之前的Attention概念类似，这里的attention仍然是学习如何从大量信息中筛选出和当前query最相关的信息，权重越大的value说明越重要。然后attention机制的具体计算过程如下图：

![](https://shangzhih.github.io/images/2018-03-30-17-39-16.jpg)

第一个阶段，将Query和每一个key进行计算F(Query,Keyi)，得到该query和每个value的相关性si。这个相关性有很多种求法，下面列举一些：

- dot

$f(Q, K_i) = Q^TK_i$，这种方法是直接将Query和每个Ki做向量点乘，注意到，这样就必须要求两者的向量维度是一致的。

- general

$f(Q, K_i) = Q^T W_a K_i$，因为引入了一个中间矩阵Wa，就不再强制要求Q和Ki的向量维度一致，这也是被称为general的原因。 - concat

$f(Q, K_i) = W_a[Q;K_i]$,先将两者进行拼接，再和Wa求积。 - perceptron

$f(Q, K_i) =v^T_a tanh(W_a Q + U_aK_i)$，这里用了三个矩阵和tanh函数。

第二个阶段就是将第一个阶段得到的相关性，进行softmax归一化，使其符合概率分布的形式ai。

$a_i = softmax(f(Q, K_i)) = \frac{exp(f(Q, K_i)}{\sum_j exp(f(Q, K_j))}$

第三个阶段就是根据第二个阶段的权重系数ai，对所有valuei进行加权求和：

$Attention(Query, Source) = \sum_{i=1}^L a_i \cdot Value_i$

目前绝大多数的使用attention的方法都可以抽象为这三个过程。

